### Melon100 Chart 
* 100곡의 노래의 제목과 SongID 추출해서 list에 저장하기
* 100곡 노래의 상세정보를 추출해서 list와 dict에 저장해서 json 파일로 저장하기
* json 파일을 load하여 Pandas의 DataFrame에 저장하기
* DataFrame 객체를 DB의 Table에 저장하기

In [1]:
import requests
from bs4 import BeautifulSoup
import re

In [50]:
url = 'https://www.melon.com/chart/index.htm'
req_header_dict = {
    # 요청헤더 : 브라우저정보
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.99 Safari/537.36'
}
res = requests.get(url, headers=req_header_dict)
print(res.status_code)

if res.ok:
    html = res.text
    soup = BeautifulSoup(html, 'html.parser')
    #<div id="tb_list"><tr><a>
    print(len(soup.select("div#tb_list tr a[href*='playSong']")))   
    a_tags = soup.select("div#tb_list tr a[href*='playSong']")
    song_list = []
    for idx,a_tag in enumerate(a_tags,1):
        song_dict = {}
        #<a href="">노래제목</a>
        song_title = a_tag.text
        song_dict['song_title'] = song_title
        
        href_value = a_tag['href']
        #Song ID를 찾기 위한 정규표현식
        matched = re.search(r'(\d+)\);', href_value)
        if matched:
            song_id = matched.group(1)
            song_dict['song_id'] = song_id
            song_detail_url = f'https://www.melon.com/song/detail.htm?songId={song_id}'
            song_dict['song_detail_url'] = song_detail_url
            #print(song_dict)
            song_list.append(song_dict)

print(len(song_list))    
print(song_list[0])

200
100
100
{'song_title': '사랑은 늘 도망가', 'song_id': '34061322', 'song_detail_url': 'https://www.melon.com/song/detail.htm?songId=34061322'}


In [68]:
import requests
from bs4 import BeautifulSoup
import re

req_header_dict = {
    # 요청헤더 : 브라우저정보
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.99 Safari/537.36'
}

song_detail_list = []
for idx,song in enumerate(song_list,1):
    #노래1곡의 상세정보를 저장할 dict 
    song_detail_dict = {}
    song_detail_url = song['song_detail_url']
    
    res = requests.get(song_detail_url, headers=req_header_dict)    
    print(res.status_code)
    if res.ok:
        soup = BeautifulSoup(res.text, 'html.parser')
        print(idx, song['song_title'])
        song_detail_dict['곡명'] = song['song_title']
        # [<span>임영웅</span>, <span></span>]
        singer_span = soup.select("a[href*='goArtistDetail'] span")
        if singer_span:
            song_detail_dict['가수'] = singer_span[0].text
        
        #print(soup.select("div.meta dd"))
        song_dd = soup.select("div.meta dd")
        if song_dd:
            #print(song_dd[0].text)
            song_detail_dict['앨범'] = song_dd[0].text
            song_detail_dict['발매일'] = song_dd[1].text
            song_detail_dict['장르'] = song_dd[2].text
        
        song_id = song['song_id']
        like_url = f'https://www.melon.com/commonlike/getSongLike.json?contsIds={song_id}'
        like_res = requests.get(like_url, headers=req_header_dict)
        if like_res.ok:
            #print(like_res.status_code)
            #print(like_res.json()['contsLike'][0]['SUMMCNT'])
            song_detail_dict['좋아요'] = like_res.json()['contsLike'][0]['SUMMCNT']
            
        #<div class="lyric" id="d_video_summary">
        lyric_div = soup.select("div#d_video_summary")
        if lyric_div:
            lyric_temp = lyric_div[0].text
            pattern = re.compile(r'[\r\n\t]')
            # \r\n\t 특수문자를 ''(empty string)으로 대체(substitute)해라
            lyric = pattern.sub('', lyric_temp.strip())
        else:
            lyric = ''
        song_detail_dict['가사'] = lyric
        
        #print(song_detail_dict)
        song_detail_list.append(song_detail_dict)

print(len(song_detail_list))
song_detail_list[:3]

200
1 사랑은 늘 도망가
200
2 취중고백
200
3 호랑수월가
200
4 Step Back
200
5 회전목마 (Feat. Zion.T, 원슈타인) (Prod. Slom)
200
6 ELEVEN
200
7 리무진 (Feat. MINO) (Prod. GRAY)
200
8 Counting Stars (Feat. Beenzino)
200
9 Dreams Come True
200
10 눈이 오잖아(Feat.헤이즈)
200
11 이제 나만 믿어요
200
12 Can't Control Myself
200
13 겨울잠
200
14 다정히 내 이름을 부르면
200
15 신호등
200
16 만남은 쉽고 이별은 어려워 (Feat. Leellamarz) (Prod. TOIL)
200
17 STAY
200
18 다시 사랑한다면 (김필 Ver.)
200
19 그대라는 사치
200
20 Next Level
200
21 strawberry moon
200
22 SMILEY (Feat. BIBI)
200
23 Christmas Tree
200
24 별빛 같은 나의 사랑아
200
25 드라마
200
26 잊었니
200
27 Savage
200
28 언덕나무
200
29 HERO
200
30 끝사랑
200
31 우리가 헤어져야 했던 이유
200
32 서랍
200
33 Bk Love
200
34 흰눈
200
35 너 아니면 안돼
200
36 Butter
200
37 너를 생각해
200
38 사랑이 아니었다고 말하지 마요
200
39 abcdefu
200
40 Child
200
41 OHAYO MY NIGHT
200
42 Dynamite
200
43 계단말고 엘리베이터
200
44 네가 없는 밤 (Feat. ASH ISLAND) (Prod. GRAY)
200
45 불협화음 (Feat. AKMU) (Prod. GRAY)
200
46 Permission to Dance
200
47 문득
200
48 여름비
200
49 바라만 본다
200
50 My Universe
200
51 Weekend


[{'곡명': '사랑은 늘 도망가',
  '가수': '임영웅',
  '앨범': '신사와 아가씨 OST Part.2',
  '발매일': '2021.10.11',
  '장르': '발라드, 국내드라마',
  '좋아요': 123988,
  '가사': '눈물이 난다 이 길을 걸으면그 사람 손길이 자꾸 생각이 난다붙잡지 못하고 가슴만 떨었지내 아름답던 사람아사랑이란 게 참 쓰린 거더라잡으려 할수록 더 멀어지더라이별이란 게 참 쉬운 거더라내 잊지 못할 사람아사랑아 왜 도망가수줍은 아이처럼행여 놓아버릴까 봐꼭 움켜쥐지만그리움이 쫓아 사랑은 늘 도망가잠시 쉬어가면 좋을 텐데바람이 분다 옷깃을 세워도차가운 이별의 눈물이 차올라잊지 못해서 가슴에 사무친내 소중했던 사람아사랑아 왜 도망가수줍은 아이처럼행여 놓아버릴까 봐꼭 움켜쥐지만그리움이 쫓아 사랑은 늘 도망가잠시 쉬어가면 좋을 텐데기다림도 애태움도 다 버려야 하는데무얼 찾아 이 길을 서성일까무얼 찾아 여기 있나사랑아 왜 도망가수줍은 아이처럼행여 놓아버릴까 봐꼭 움켜쥐지만그리움이 쫓아 사랑은 늘 도망가잠시 쉬어가면 좋을 텐데잠시 쉬어가면 좋을 텐데'},
 {'곡명': '취중고백',
  '가수': '김민석 (멜로망스)',
  '앨범': '취중고백',
  '발매일': '2021.12.19',
  '장르': '발라드',
  '좋아요': 68683,
  '가사': '뭐하고 있었니 늦었지만잠시 나올래너의 집 골목에 있는놀이터에 앉아 있어친구들 만나서 오랜만에술을 좀 했는데자꾸만 니 얼굴 떠올라무작정 달려왔어이 맘 모르겠니요즘 난 미친 사람처럼너만 생각해대책없이 네가 점점 좋아져아냐 안 취했어 진짜야널 정말 사랑해눈물이 날만큼 원하고 있어정말로 몰랐니가끔 전화해 장난치듯주말엔 뭐할거냐며너의 관심 끌던 나를그리고 한번씩 누나 주려 샀는데너 그냥 준다고생색 낸 선물도 너 때문에 산거야이 맘 모르겠니요즘 난 미친 사람처럼너만 생각해대책없이 네가 점점 좋아져아냐 안 취했어 진짜야널 정말 사랑해진심이야 믿어줘갑자기 이런 말 놀랐다면 미안해부담이 되는

In [73]:
import json

with open('data/songs.json','w', encoding='utf-8') as file:
    json.dump(song_detail_list, file)

In [76]:
with open('data/songs.json',encoding='utf-8') as file:
    song_json= json.loads(file.read())

#print(song_json)    

### Pandas 의 DataFrame 객체 사용하기

In [89]:
#song_detail_list을 읽어서 DataFrame 객체를 생성하는 방법
#DataFrame 객체를 생성
import pandas as pd

song_df2 = pd.DataFrame(columns=['곡명','가수','앨범','발매일','장르','좋아요','가사'])

#1개의 row = Series 객체, 1개의 column = Series 객체
for song_detail in song_detail_list:
    # dict <=> Series    
    series_obj = pd.Series(song_detail)
    # list <=> DataFrame
    song_df2 = song_df2.append(series_obj, ignore_index=True)

song_df2.head()    

,곡명,가수,앨범,발매일,장르,좋아요,가사
0,사랑은 늘 도망가,임영웅,신사와 아가씨 OST Part.2,2021.10.11,"발라드, 국내드라마",123988,눈물이 난다 이 길을 걸으면그 사람 손길이 자꾸 생각이 난다붙잡지 못하고 가슴만 떨...
1,취중고백,김민석 (멜로망스),취중고백,2021.12.19,발라드,68683,뭐하고 있었니 늦었지만잠시 나올래너의 집 골목에 있는놀이터에 앉아 있어친구들 만나서...
2,호랑수월가,탑현,호랑수월가,2022.01.15,발라드,35282,흐르는 저 하늘을 물어 채는 범처럼태산에 날아들어 숨어드는 새처럼동산을 뛰고 뛰어가...
3,Step Back,GOT the beat,Step Back,2022.01.03,댄스,65658,You must step back어델 어델 봐너 감히 누구라고 날 제껴이쯤에서 물러...
4,"회전목마 (Feat. Zion.T, 원슈타인) (Prod. Slom)",sokodomo,쇼미더머니 10 Episode 2,2021.11.13,랩/힙합,186558,내가 슬플 때마다이 노래가 찾아와세상이 둥근 것처럼 우린 동글동글인생은 회전목마우린...


In [81]:
import pandas as pd

#json file을 읽어서 DataFrame 객체를 생성하는 방법
song_df = pd.read_json('data/songs.json')
print(type(song_df))
song_df.head()

<class 'pandas.core.frame.DataFrame'>


,곡명,가수,앨범,발매일,장르,좋아요,가사
0,사랑은 늘 도망가,임영웅,신사와 아가씨 OST Part.2,2021.10.11,"발라드, 국내드라마",123988,눈물이 난다 이 길을 걸으면그 사람 손길이 자꾸 생각이 난다붙잡지 못하고 가슴만 떨...
1,취중고백,김민석 (멜로망스),취중고백,2021.12.19,발라드,68683,뭐하고 있었니 늦었지만잠시 나올래너의 집 골목에 있는놀이터에 앉아 있어친구들 만나서...
2,호랑수월가,탑현,호랑수월가,2022.01.15,발라드,35282,흐르는 저 하늘을 물어 채는 범처럼태산에 날아들어 숨어드는 새처럼동산을 뛰고 뛰어가...
3,Step Back,GOT the beat,Step Back,2022.01.03,댄스,65658,You must step back어델 어델 봐너 감히 누구라고 날 제껴이쯤에서 물러...
4,"회전목마 (Feat. Zion.T, 원슈타인) (Prod. Slom)",sokodomo,쇼미더머니 10 Episode 2,2021.11.13,랩/힙합,186558,내가 슬플 때마다이 노래가 찾아와세상이 둥근 것처럼 우린 동글동글인생은 회전목마우린...


In [90]:
song_df.tail()

,곡명,가수,앨범,발매일,장르,좋아요,가사
95,좋아좋아,조정석,슬기로운 의사생활 시즌2 OST Part 5,2021.07.16,"발라드, 국내드라마",83250,처음 널 만나는 날노란 세 송이 장미를 들고룰루랄라 신촌을 향하는내 가슴은 마냥 두...
96,나 그댈위해 시 한편을 쓰겠어,케이시 (Kassy),옛이야기,2021.10.28,발라드,31440,나 그댈 위해 시 한 편을 쓰겠어나 그대에게 못다 한 얘기 많아차마 그때 하지 못 ...
97,깨우지 마,MC몽,X by X [ 꿈 ],2022.01.21,랩/힙합,7704,깨우지 마 꿈속으로도망치듯 잠에 든 날하루 중 내가 유일하게무책임해도 되는 시간날 ...
98,Still I Love You,토요태,Still I Love You,2021.12.18,R&B/Soul,31471,여보세요어 잘 지내지오빠 전화하지 말랬잖아아 그치 어우 야 밖에 많이 춥다따뜻하게 ...
99,MBTI (Feat. 쿠기 & 로꼬) (Prod. GRAY),BE'O (비오),쇼미더머니 10 Semi Final,2021.11.27,랩/힙합,45093,Million 내 몸값을 더 올려Billion 내 몸값을 더 올려Trillion 내...


In [97]:
print('shape', song_df.shape)
print('columns', song_df.columns)
print('index', song_df.index)
print('values', type(song_df.values)) #2차원배열

shape (100, 7)
columns Index(['곡명', '가수', '앨범', '발매일', '장르', '좋아요', '가사'], dtype='object')
index RangeIndex(start=0, stop=100, step=1)
values <class 'numpy.ndarray'>


In [99]:
song_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   곡명      100 non-null    object
 1   가수      100 non-null    object
 2   앨범      100 non-null    object
 3   발매일     100 non-null    object
 4   장르      100 non-null    object
 5   좋아요     100 non-null    int64 
 6   가사      100 non-null    object
dtypes: int64(1), object(6)
memory usage: 5.6+ KB


In [102]:
# 가수 컬럼의 값을 선택하기
print(type(song_df['가수']))
song_df['가수'].head()

<class 'pandas.core.series.Series'>


0             임영웅
1      김민석 (멜로망스)
2              탑현
3    GOT the beat
4        sokodomo
Name: 가수, dtype: object

In [104]:
# 가수 컬럼의 값별로 Row Counting
song_df['가수'].value_counts()

임영웅                    10
아이유                     7
방탄소년단                   6
BE'O (비오)               5
aespa                   3
                       ..
Coldplay                1
아넌딜라이트(Anandelight)     1
김민석 (멜로망스)              1
쿠기 (Coogie)             1
토요태                     1
Name: 가수, Length: 65, dtype: int64

In [105]:
# 장르 컬럼의 값별로 row counting
song_df['장르'].value_counts()

발라드               24
랩/힙합              21
댄스                18
발라드, 국내드라마        11
성인가요               7
POP                6
록/메탈               4
R&B/Soul           4
R&B/Soul, 인디음악     3
포크/블루스, 국내드라마      1
발라드, 인디음악          1
Name: 장르, dtype: int64

In [107]:
# unique한(중복제거) 가수 컬럼의 값을 가져오기
song_df['가수'].unique()

array(['임영웅', '김민석 (멜로망스)', '탑현', 'GOT the beat', 'sokodomo', 'IVE (아이브)',
       "BE'O (비오)", 'aespa', '이무진', '태연 (TAEYEON)', '아이유', '경서예지',
       '베이식 (Basick)', 'The Kid LAROI', 'YENA (최예나)', 'V', '이승윤',
       '비비 (BIBI)', '10CM', '먼데이 키즈 (Monday Kiz)', '방탄소년단',
       '주시크 (Joosiq)', '이예준', 'GAYLE', '마크 (MARK)', '디핵 (D-Hack)',
       'Mudd the student', '샘김 (Sam Kim)', 'MSG워너비(M.O.M)', 'Coldplay',
       '아넌딜라이트(Anandelight)', 'AKMU (악뮤)', '개코', '멜로망스', '쿠기 (Coogie)',
       '김나영', '경서', '헤이즈 (Heize)', 'Ed Sheeran', '호미들', '황인욱',
       '마크툽 (MAKTUB)', '폴킴', '프로미스나인', '선예', 'Justin Bieber', '전상근',
       '성시경', '정동하', 'ASH ISLAND', '브레이브걸스', '오마이걸 (OH MY GIRL)', '송이한',
       'TOIL', '산들', '케이윌', '휘인 (Whee In)', 'Jawsh 685',
       'Red Velvet (레드벨벳)', 'Kep1er (케플러)', '잠골버스', '조정석', '케이시 (Kassy)',
       'MC몽', '토요태'], dtype=object)

In [108]:
song_df['장르'].unique()

array(['발라드, 국내드라마', '발라드', '댄스', '랩/힙합', '성인가요', '록/메탈', 'POP',
       'R&B/Soul, 인디음악', '포크/블루스, 국내드라마', 'R&B/Soul', '발라드, 인디음악'],
      dtype=object)

In [110]:
# 가사 empty string check
song_df.loc[song_df['가사'] == '']

,곡명,가수,앨범,발매일,장르,좋아요,가사


In [112]:
# 방탄소년단 노래 선택하기
song_df['가수'] == '방탄소년단'

0     False
1     False
2     False
3     False
4     False
      ...  
95    False
96    False
97    False
98    False
99    False
Name: 가수, Length: 100, dtype: bool

In [113]:
song_df.loc[song_df['가수'] == '방탄소년단']

,곡명,가수,앨범,발매일,장르,좋아요,가사
35,Butter,방탄소년단,Butter,2021.05.21,댄스,263255,Smooth like butterLike a criminal undercoverGo...
41,Dynamite,방탄소년단,Dynamite (DayTime Version),2020.08.24,댄스,428692,Cos ah ahI’m in the stars tonightSo watch me b...
45,Permission to Dance,방탄소년단,Butter / Permission to Dance,2021.07.09,댄스,195014,It’s the thought of being youngWhen your heart...
77,봄날,방탄소년단,YOU NEVER WALK ALONE,2017.02.13,랩/힙합,555844,보고 싶다이렇게 말하니까 더 보고 싶다너희 사진을 보고 있어도보고 싶다너무 야속한 ...
82,작은 것들을 위한 시 (Boy With Luv) (Feat. Halsey),방탄소년단,MAP OF THE SOUL : PERSONA,2019.04.12,댄스,431668,모든 게 궁금해How’s your dayOh tell me뭐가 널 행복하게 하는지O...
94,Life Goes On,방탄소년단,BE,2020.11.20,랩/힙합,233287,어느 날 세상이 멈췄어아무런 예고도 하나 없이봄은 기다림을 몰라서눈치 없이 와버렸어...


In [119]:
def get_song(singer):
    return song_df.loc[song_df['가수'] == singer, '곡명':'좋아요']

In [120]:
get_song('임영웅')

,곡명,가수,앨범,발매일,장르,좋아요
0,사랑은 늘 도망가,임영웅,신사와 아가씨 OST Part.2,2021.10.11,"발라드, 국내드라마",123988
10,이제 나만 믿어요,임영웅,내일은 미스터트롯 우승자 특전곡,2020.04.03,성인가요,146420
17,다시 사랑한다면 (김필 Ver.),임영웅,사랑의 콜센타 PART56,2021.05.25,성인가요,40773
18,그대라는 사치,임영웅,사랑의 콜센타 PART62,2021.07.06,발라드,36831
23,별빛 같은 나의 사랑아,임영웅,별빛 같은 나의 사랑아,2021.03.09,성인가요,69703
25,잊었니,임영웅,사랑의 콜센타 PART60,2021.06.22,성인가요,36020
28,HERO,임영웅,HERO,2020.11.04,성인가요,69936
29,끝사랑,임영웅,사랑의 콜센타 PART61,2021.06.29,성인가요,33317
32,Bk Love,임영웅,뽕숭아학당 PART23,2021.06.21,랩/힙합,33012
42,계단말고 엘리베이터,임영웅,계단말고 엘리베이터,2018.08.20,성인가요,56703


#### 특정 행과 열을 선택하기
* loc[], iloc[] 사용한다.
* 1. Slicing 을 사용해서 구간을 주어서 행과 열을 선택
* 2. <pre>[ ]</pre> (list)를 사용해서 여러개의 행과 열을 선택
* 3. 조건식을 만족하는 행과 열을 선택

In [ ]:
# Slicing 인덱스가 0부터 5까지의 행과 모든 열을 선택하기
song_df